# Applied Data Science Capstone
## Project Title: The Battle of Neighborhoods

Introduction:
The project is trying to find out the difference between Hong Kong and one of the BIG FOUR city in Maindland china, since most of conversions between Hongkongers and Chinese mentioned how great are their life and their diversity of choices to live.


Data:  
Data will of coures come from FourSquare API, making use of the explore function, we will make clusters for cities and compare the top 10 common venues of theirs.


In [1]:
#import libraries
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
C_ID = 'NSNZ2FAIXDEOCGAZGQ2LBMBV3YMQL1X0J1KHFE1UA40R0SEB'
C_SC = 'KFLH2TZRLVJXWMOLWVBDEGPBUFHS3OMGGH5BQRVV2PG5EECS'
V = 20200509
L = 100
R = 500

In [3]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(C_ID,C_SC,V,test_lat,test_lng,R,L)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NSNZ2FAIXDEOCGAZGQ2LBMBV3YMQL1X0J1KHFE1UA40R0SEB&client_secret=KFLH2TZRLVJXWMOLWVBDEGPBUFHS3OMGGH5BQRVV2PG5EECS&v=20200509&ll=31.28333,121.1622&radius=500&limit=100'

In [4]:
results = requests.get(url).json()

In [5]:
def getCategoryType(row):
    try:
        cate_list = row['categories']
    except:
        cate_list = row['venue.categories']
    if len(cate_list) == 0:
        return None
    else:
        return cate_list[0]['name'] 

In [6]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(getCategoryType, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,上海汽车博物馆,History Museum,31.280813,121.165556
1,KFC (肯德基),Fast Food Restaurant,31.282988,121.162613
2,Shanghai Auto Expo Park (上海汽车博览公园),Park,31.281811,121.163765
3,Shanghai Auto Museum,Science Museum,31.280883,121.165432


In [24]:
#get a list of neighborhoods on SH
#get the wiki HTML and read it in soup
from bs4 import BeautifulSoup
import urllib
url = 'https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Shanghai'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')

In [35]:
#find targeted class of content
table = soup.find('div',{'class':'mw-content-ltr'})
#find all links
allLinks = table.findAll('a')
#expell all useless link
all_nei_links = allLinks[13:]
all_nei_links
#store links title to a list
sh_nei = []
for link in all_nei_links:
    sh_nei.append(link.get('title'))
#done
#sh_nei
df_sh = pd.DataFrame()
df_sh['Neighborhood'] = sh_nei
df_sh

,Neighborhood
0,Anting
1,Changshou Road Subdistrict
2,Fengjing
3,"Gaoqiao, Shanghai"
4,"Gubei, Shanghai"
5,"Koreatown, Shanghai"
6,Lujiazui
7,"Luodian, Shanghai"
8,Nanxiang
9,Qiantan International Business Zone (Shanghai)


In [ ]:
#get neighborhood coordinates and append to df_sh, if no result, then return nan
def getCoord()
